In [23]:
# 2018 Kaggle ML & DS Survey Challenge
## Autor: Jairo Andres Saavedra Alfonso 
## Universidad de Los Andes 
## Primera para Final de Metodos Computacionales Avanzados

## ¿Que tan bueno para la economia de un estudiante es saber de ML en latinoamerica?
### Para responder esta pregunta se trato mediante diferentes tecnicas de ML para ver si se puede ajustar un modelo mediante los datos disponibles en el 2018 Kaggle ML & DS Survey Challenge. 
### El modelo que mejor se ajuste nos dara un aproximado de cuanto puede ganar una persona en Latinoamerica dependiendo de: su sexo, Edad, Ultimo Nivel academico logrado, Paiz, Numero de Años de Experiencia y Numero de lenguajes de programacion que maneja

In [24]:
### Primero que todo se realiza la debida importacion de las librerias a usar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.model_selection
import sklearn.preprocessing
import scipy.integrate
import sklearn.discriminant_analysis
import sklearn.ensemble
import sklearn.tree
import itertools
import sklearn.tree
import sklearn.metrics


In [25]:
### Se cargan los datos a trabajar los cuales son multipleChoiceResponses.csv extraidos de https://www.kaggle.com/kaggle/kaggle-survey-2018#multipleChoiceResponses.csv
names=['Q1','Q2','Q3','Q4','Q8','Q9','Q16_Part_1','Q16_Part_2','Q16_Part_3','Q16_Part_4','Q16_Part_5','Q16_Part_6','Q16_Part_7','Q16_Part_8','Q16_Part_9','Q16_Part_10','Q16_Part_11','Q16_Part_12','Q16_Part_13','Q16_Part_14','Q16_Part_15','Q16_Part_16'
,'Q16_Part_18']
data=pd.read_csv('multipleChoiceResponses.csv',usecols=names)
data=data.iloc[1:,:]
print(data)

                      Q1     Q2  \
1                 Female  45-49   
2                   Male  30-34   
3                 Female  30-34   
4                   Male  35-39   
5                   Male  22-24   
6                   Male  25-29   
7                   Male  35-39   
8                   Male  18-21   
9                   Male  25-29   
10                  Male  30-34   
11                  Male  22-24   
12                  Male  40-44   
13                  Male  25-29   
14                  Male  25-29   
15                Female  25-29   
16                  Male  25-29   
17                  Male  40-44   
18                  Male  25-29   
19                  Male  35-39   
20                  Male  55-59   
21                Female  30-34   
22                Female  25-29   
23                  Male  30-34   
24                Female  25-29   
25                  Male  30-34   
26                  Male  25-29   
27                  Male  30-34   
28                Fe

In [14]:
### Se realiza la separacion de los paices Latinoamericanos
def latin_countries(data):
    dataC=data[data['Q3']=='Colombia']
    dataCh=data[data['Q3']=='Chile']
    dataP=data[data['Q3']=='Peru']
    dataB=data[data['Q3']=='Brazil']
    dataAr=data[data['Q3']=='Argentina']
    dataMx=data[data['Q3']=='Mexico']

    # Concatening all countries on one matrix
    frames=[dataC,dataCh,dataP,dataB,dataAr,dataMx]
    new_data= pd.concat(frames)
    return new_data
new_data=latin_countries(data)

In [32]:
### Se eliminan de Data los datos que presenten irregularidad o los que no contribuyen a el proposito 
def nans_killer(new_data):
    new_data=new_data.drop(new_data[new_data['Q1']=='Prefer to self-describe'].index)   
    new_data=new_data.drop(new_data[new_data['Q1']=='Prefer not to say'].index)
    new_data=new_data.drop(new_data[new_data['Q4']=='I prefer not to answer'].index)
    new_data=new_data.dropna(axis=0,subset=['Q4'])
    new_data=new_data.dropna(axis=0,subset=['Q8'])
    new_data=new_data.dropna(axis=0,subset=['Q9'])
    new_data=new_data.drop(new_data[new_data['Q9']=='I do not wish to disclose my approximate yearly compensation'].index)
    return new_data
new_data=nans_killer(new_data)
print(new_data)

          Q1     Q2        Q3  \
17      Male  40-44  Colombia   
1609    Male  25-29  Colombia   
1611    Male  60-69  Colombia   
1725    Male  22-24  Colombia   
1780    Male  25-29  Colombia   
1946    Male  30-34  Colombia   
1977    Male  30-34  Colombia   
2097    Male  35-39  Colombia   
2163    Male  25-29  Colombia   
2197    Male  30-34  Colombia   
2246    Male  30-34  Colombia   
2511    Male  25-29  Colombia   
2581    Male  30-34  Colombia   
2811  Female  22-24  Colombia   
2842    Male  25-29  Colombia   
3438    Male  25-29  Colombia   
3865    Male  30-34  Colombia   
3990    Male  22-24  Colombia   
4224    Male  30-34  Colombia   
4450    Male  30-34  Colombia   
4494    Male  25-29  Colombia   
4675    Male  35-39  Colombia   
4676    Male  40-44  Colombia   
7       Male  35-39     Chile   
1121  Female  25-29     Chile   
1568    Male  45-49     Chile   
1771    Male  25-29     Chile   
1887    Male  45-49     Chile   
2092    Male  22-24     Chile   
2102    Ma

In [88]:
### Se concatena la matriz final de datos a trabajar
def matrix(data):
    data=data.fillna(0)
    data=data.values
    data=data.T
    #print(data)
    rows=len(data[0])

    matrix=np.zeros((7,rows))
    
    ii=data[0]=='Male'
    matrix[0,ii]=1.0
    matrix[0,~ii]=-1.0
    
    i1=data[1]=='18-21'
    i2=data[1]=='22-24'
    i3=data[1]=='25-29'
    i4=data[1]=='30-34'
    i5=data[1]=='35-39'
    i6=data[1]=='40-44'
    i7=data[1]=='45-49'
    i8=data[1]=='50-54'
    i9=data[1]=='55-59'
    i10=data[1]=='60-69'
    i11=data[1]=='70-79'
    i12=data[1]=='80+'
    matrix[1,i1]=np.mean([18,21])
    matrix[1,i2]=np.mean([22,24])
    matrix[1,i3]=np.mean([25,29])
    matrix[1,i4]=np.mean([30,34])
    matrix[1,i5]=np.mean([35,39])
    matrix[1,i6]=np.mean([40,44])
    matrix[1,i7]=np.mean([45,49])
    matrix[1,i8]=np.mean([50,54])
    matrix[1,i9]=np.mean([55,59])
    matrix[1,i10]=np.mean([60,69])
    matrix[1,i11]=np.mean([70,79])
    matrix[1,i12]=80.0
    
    o1=data[2]=='Colombia'
    o2=data[2]=='Chile'
    o3=data[2]=='Peru'
    o4=data[2]=='Brazil'
    o5=data[2]=='Argentica'
    o6=data[2]=='Mexico'
    matrix[2,o1]=10 
    matrix[2,o2]=20
    matrix[2,o3]=30
    matrix[2,o4]=40
    matrix[2,o5]=50
    matrix[2,o6]=60
    
    a1=data[3]=='Some college/university study without earning a bachelor’s degree'
    a2=data[3]=='Professional degree'
    a3=data[3]=='Bachelor’s degree'
    a4=data[3]=='Master’s degree'    
    a5=data[3]=='Doctoral degree'
    matrix[3,a1]=100 
    matrix[3,a2]=200
    matrix[3,a3]=300
    matrix[3,a4]=400
    matrix[3,a5]=500
    
    a1=data[5]=='0-10,000'
    a2=data[5]=='10-20,000'
    a3=data[5]=='20-30,000'
    a4=data[5]=='30-40,000'
    a5=data[5]=='40-50,000'
    a6=data[5]=='50-60,000'
    a7=data[5]=='60-70,000'
    a8=data[5]=='70-80,000'
    a9=data[5]=='80-90,000'
    a10=data[5]=='90-100,000'
    a11=data[5]=='100-125,000'
    a12=data[5]=='125-150,000'
    a13=data[5]=='150-200,000'
    a14=data[5]=='200-250,000'
    a15=data[5]=='250-300,000'
    a16=data[5]=='300-400,000'
    a17=data[5]=='400-500,000'
    a18=data[5]=='500,000+'
    matrix[4,a1]=786
    matrix[4,a2]=865
    matrix[4,a3]=322
    matrix[4,a4]=124
    matrix[4,a5]=325
    matrix[4,a6]=741
    matrix[4,a7]=965
    matrix[4,a8]=564
    matrix[4,a9]=624
    matrix[4,a10]=523
    matrix[4,a11]=693
    matrix[4,a12]=800
    matrix[4,a13]=738
    matrix[4,a14]=239
    matrix[4,a15]=469
    matrix[4,a16]=697
    matrix[4,a17]=108
    matrix[4,a18]=809
    
    a1=data[4]=='0-1'
    a2=data[4]=='1-2'
    a3=data[4]=='2-3'
    a4=data[4]=='3-4'
    a5=data[4]=='4-5'
    a6=data[4]=='5-10'
    a7=data[4]=='10-15'
    a8=data[4]=='15-20'
    a9=data[4]=='20-25'
    a10=data[4]=='25-30'
    a11=data[4]=='30+'
    matrix[5,a1]=0.5
    matrix[5,a2]=1.5
    matrix[5,a3]=2.5
    matrix[5,a4]=3.5
    matrix[5,a5]=4.5
    matrix[5,a6]=7.5
    matrix[5,a7]=12.5
    matrix[5,a8]=17.5
    matrix[5,a9]=22.5
    matrix[5,a10]=27.5
    matrix[5,a11]=30
    rang=np.arange(8,22,1)
    for j in range(rows):
        cont=0
        for i in rang:
            if(data[i,j]!=0):
                cont=cont+1
        matrix[6,j]=cont
    

    
    return matrix.T
matrix=matrix(new_data)
index=np.arange(0,265,1)
columns=np.arange(0,7,1)
Data=pd.DataFrame(matrix,index=index,columns=columns)

In [89]:
y=Data[4]
Data = Data.drop([4],axis=1)

predictors = list(Data.keys())
train_data, test_data , y_train , y_test = sklearn.model_selection.train_test_split(Data,y, train_size=0.5)
modelos = {}

/srv/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [90]:
### Usando LR
model_lr = sklearn.linear_model.LogisticRegression()
model_lr.fit(train_data[predictors], y_train)
modelos['LR'] = model_lr

/srv/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/srv/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [91]:
### Usando LDA
model_lda = sklearn.discriminant_analysis.LinearDiscriminantAnalysis()
model_lda.fit(train_data[predictors], y_train)
modelos['LDA'] = model_lda


In [94]:
for k in modelos.keys():
    probas = modelos[k].predict_proba(test_data[predictors])
    precision, recall, _ = sklearn.metrics.precision_recall_curve(y_test, probas[:,1])
    ii = np.argsort(recall)
    area = scipy.integrate.trapz(precision[ii], recall[ii])
    plt.plot(recall, precision, label=k+' area: {:.3f}'.format(area))
plt.legend()
plt.xlabel('Recall')
plt.ylabel('Precision')


AttributeError: 'DataFrame' object has no attribute 'Target'